## Demo: Stage2 processing for mooring data

- Apply clock offsets
- Trim to deployment period

In [2]:
import os

import yaml
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from datetime import datetime


In [10]:
# Specify the base directory.  raw is a subdirectory from here moor/raw/ and proc is moor/proc
basedir = '/Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/'
output_path = basedir + 'moor/proc/'

moorlist = ['ds2_X_2012','ds2_X_2017','ds2_X_2018',
            'ds8_1_2012','ds9_1_2012','ds10_1_2012', 'ds11_1_2012','ds12_1_2012',
            'ds13_1_2012','ds14_1_2012','ds15_1_2012','ds16_1_2012','ds17_1_2012',
            'ds19_1_2012','ds18_1_2012','ds28_1_2017',
            'dsA_1_2018','dsB_1_2018','dsC_1_2018', 'dsD_1_2018','dsE_1_2018','dsF_1_2018',
            'dsM1_1_2017','dsM2_1_2017','dsM3_1_2017','dsM4_1_2017','dsM5_1_2017']
moorlist = ['dsC_1_2018']

def read_yaml_time(data, key):
    """Return datetime64[ns] from YAML dict or NaT if missing/invalid."""
    val = data.get(key, None)
    if val is None or (isinstance(val, str) and not val.strip()):
        return np.datetime64("NaT", "ns")
    try:
        return pd.to_datetime(val).to_datetime64()
    except Exception:
        return np.datetime64("NaT", "ns")

# Cycle through the yaml and load instrument data into a list of xarray datasets
# Enrich the netCDF with information from the yaml file
# Find the mooring's processed directory & read the yaml specification
name1 = moorlist[0]
proc_dir = output_path + name1
moor_yaml = proc_dir + '/' + name1 + '.mooring.yaml'
with open(moor_yaml, 'r') as f:
    moor_yaml_data = yaml.safe_load(f)

# For each instrument, load the raw netCDF files and add some metadata from the yaml
datasets = []
deploy_time  = read_yaml_time(moor_yaml_data, "deployment_time")
recover_time = read_yaml_time(moor_yaml_data, "recovery_time")


for i in moor_yaml_data['instruments']:
    fname = name1 + '_' + str(i['serial']) + '_raw.nc'
    rawfile = proc_dir + '/' + i['instrument'] + '/' + fname

    if os.path.exists(rawfile):
        ds1 = xr.open_dataset(rawfile)

        if 'InstrDepth' not in ds1.variables and 'depth' in i:
            ds1['InstrDepth'] = i['depth']
        if 'instrument' not in ds1.variables and 'instrument' in i:
            ds1['instrument'] = i['instrument']
        if 'serial_number' not in ds1.variables and 'serial' in i:
            ds1['serial_number'] = i['serial']
        if 'timeS' in ds1.variables:
            ds1 = ds1.drop_vars('timeS')

        # Apply the clock offset
        ds1['clock_offset'] = i.get('clock_offset', 0)
        ds1['clock_offset'].attrs['units'] = 's'
        clock_offset = ds1['clock_offset'].values
        ds1['time'] = ds1['time'] + np.timedelta64(int(ds1['clock_offset'].values), 's')

        if np.isfinite(deploy_time):
            ds1 = ds1.sel(time=slice(deploy_time, None))
        if np.isfinite(recover_time):
            ds1 = ds1.sel(time=slice(None, recover_time))
        #---------------------------------------------
        # Store the data in a list of datasets
        fname2 = fname.replace('_raw','_use')
        print(f"Saving to {proc_dir} + {fname2}")
        ds1.to_netcdf(proc_dir + '/' + i['instrument'] + '/' + fname2)





Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6368_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_7091_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6391_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6393_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6407_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6404_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6406_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6392_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/d

/var/folders/t1/z5bp59k95119nw35yqv699t40000gn/T/ipykernel_27202/4183206822.py:43: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds1 = xr.open_dataset(rawfile)


Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_7090_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6395_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6367_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6371_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6372_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_15176_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_4774_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/dsC_1_2018 + dsC_1_2018_6408_use.nc
Saving to /Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/moor/proc/